In [3]:
import xarray as xr
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_validate
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import uniform, randint

import numpy as np
from sklearn.model_selection import LeaveOneGroupOut



In [4]:
xr_data=xr.load_dataset("../results_stacked_hh.nc")

In [5]:
xr_data

<xarray.Dataset> Size: 489kB
Dimensions:                      (sample: 100, case_index: 182)
Coordinates:
  * sample                       (sample) float64 800B 0.0 1.0 2.0 ... 98.0 99.0
    k_b                          (sample) float64 800B 0.04 0.05322 ... 0.04703
    ss_alpha                     (sample) float64 800B 0.875 0.8818 ... 0.9873
    wind_farm                    (case_index) <U31 23kB 'HR1' ... 'VirtWF_ABL...
    flow_case                    (case_index) float64 1kB 0.0 1.0 ... 14.0 15.0
Dimensions without coordinates: case_index
Data variables: (12/22)
    model_bias_cap               (sample, case_index) float64 146kB 0.04646 ....
    pw_power_cap                 (sample, case_index) float64 146kB 0.6068 .....
    ref_power_cap                (sample, case_index) float64 146kB 0.5603 .....
    wind_speed                   (case_index) float64 1kB 9.947 9.971 ... 12.97
    wind_direction               (case_index) float64 1kB 14.98 9.983 ... 270.0
    turbulence_intensity         (case_index) float64 1kB 0.0538 ... 0.03155
    ...                           ...
    nt                           (case_index) int64 1kB 80 80 80 ... 100 100 100
    farm_density                 (case_index) float64 1kB 2.452e+05 ... 1.633...
    Blockage_Ratio               (case_index) float64 1kB 0.1439 ... 0.8986
    Blocking_Distance            (case_index) float64 1kB 1.0 1.0 ... 0.326
    Farm_Length                  (case_index) float64 1kB 62.0 58.0 ... 45.0
    Farm_Width                   (case_index) float64 1kB 79.0 76.0 ... 45.0

In [7]:
xr_data

<xarray.Dataset> Size: 489kB
Dimensions:                      (sample: 100, case_index: 182)
Coordinates:
  * sample                       (sample) float64 800B 0.0 1.0 2.0 ... 98.0 99.0
    k_b                          (sample) float64 800B 0.04 0.05322 ... 0.04703
    ss_alpha                     (sample) float64 800B 0.875 0.8818 ... 0.9873
    wind_farm                    (case_index) <U31 23kB 'HR1' ... 'VirtWF_ABL...
    flow_case                    (case_index) float64 1kB 0.0 1.0 ... 14.0 15.0
Dimensions without coordinates: case_index
Data variables: (12/22)
    model_bias_cap               (sample, case_index) float64 146kB 0.04646 ....
    pw_power_cap                 (sample, case_index) float64 146kB 0.6068 .....
    ref_power_cap                (sample, case_index) float64 146kB 0.5603 .....
    wind_speed                   (case_index) float64 1kB 9.947 9.971 ... 12.97
    wind_direction               (case_index) float64 1kB 14.98 9.983 ... 270.0
    turbulence_intensity         (case_index) float64 1kB 0.0538 ... 0.03155
    ...                           ...
    nt                           (case_index) int64 1kB 80 80 80 ... 100 100 100
    farm_density                 (case_index) float64 1kB 2.452e+05 ... 1.633...
    Blockage_Ratio               (case_index) float64 1kB 0.1439 ... 0.8986
    Blocking_Distance            (case_index) float64 1kB 1.0 1.0 ... 0.326
    Farm_Length                  (case_index) float64 1kB 62.0 58.0 ... 45.0
    Farm_Width                   (case_index) float64 1kB 79.0 76.0 ... 45.0

In [6]:
X = xr_data.drop_vars(['case_index', 'wind_farm','model_bias_cap',
                     'wind_direction', 'pw_power_cap', 'ref_power_cap'])#,'nt','turb_rated_power'
y = xr_data['model_bias_cap']
validation_data=xr_data[['turb_rated_power', 'pw_power_cap', 'ref_power_cap']]
groups = xr_data['wind_farm'].values

ValueError: These variables cannot be found in this dataset: ['case_index']

In [71]:
np.argmin(np.abs(np.sum(xr_data['model_bias_cap'].values,axis=1)))


12

In [44]:
flow_cases=df['case_index'].unique()
groups=xr_data['wind_farm'].values

In [64]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)
splits = cv.split(flow_cases)


In [65]:
for train_idx, test_idx in splits:
    # print('train:', groups[train_idx])
    print('test:', groups[test_idx])


test: ['HR1' 'HR2' 'HR2' 'HR3' 'NYSTED2' 'NYSTED2' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_ali_DX5_DY5'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_ali_DX5_DY5'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_ali_DX5_DY5'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_stag_DX5_DY7p5'
 'VirtWF_ABL_IEA15_stag_DX7p5_DY5' 'VirtWF_ABL_IEA22' 'VirtWF_ABL_IEA22'
 'VirtWF_ABL_IEA22']
test: ['HR1' 'HR2' 'HR2' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED2'
 'NYSTED2' 'NYSTED2' 'NYSTED2' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_stag_DX5_DY5'
 'VirtWF_ABL_IEA15_stag_DX5_DY7p5' 'VirtWF_ABL_IEA15_stag_DX7p5_DY5'
 'VirtWF_ABL_IEA22' 'VirtWF_ABL_IEA22']
test: ['HR2' 'HR2' 'HR2' 'HR3' 'NYSTED1' 'NYSTED1' 'NYSTED2' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_ali_DX5_DY5'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_ali_DX5_DY5'
 'VirtWF_ABL_IEA15_ali_DX5_DY5' 'VirtWF_ABL_IEA15_ali_DX5_

In [11]:
df = xr_data.to_dataframe().reset_index()
        # df['log_z0'] = np.log(df['z0'].values)

X = df.drop(['case_index', 'wind_farm', 'flow_case','model_bias_cap',
                'wind_direction', 'pw_power_cap', 'ref_power_cap'], axis=1)#,'nt','turb_rated_power'
y = df[['model_bias_cap']]

In [12]:
X

,sample,wind_speed,turbulence_intensity,potential_temperature,epsilon,k,lapse_rate,z0,LMO,ABL_height,...,capping_inversion_thickness,turb_rated_power,nt,farm_density,k_b,ss_alpha,Blockage_Ratio,Blocking_Distance,Farm_Length,Farm_Width
0,0.0,9.946975,0.053803,293.157143,0.001880,0.429605,0.001008,0.002090,100000.0,951.166896,...,274.327597,2000000.0,80,2.451599e+05,0.040000,0.875000,0.143910,1.000000,62.0,79.0
1,0.0,9.971009,0.069163,293.164520,0.003978,0.713338,0.001018,0.020800,100000.0,956.370264,...,232.729528,2000000.0,80,2.451599e+05,0.040000,0.875000,0.112734,1.000000,58.0,76.0
2,0.0,9.952359,0.060814,293.160126,0.002704,0.549462,0.001015,0.006864,100000.0,952.575951,...,253.596873,2000000.0,80,2.451599e+05,0.040000,0.875000,0.212993,1.000000,53.0,73.0
3,0.0,9.955688,0.057949,293.158852,0.002347,0.499248,0.001013,0.004365,100000.0,951.768487,...,262.045921,2000000.0,80,2.451599e+05,0.040000,0.875000,0.058441,0.961863,49.0,69.0
4,0.0,9.950229,0.058088,293.158852,0.002360,0.501094,0.001013,0.004467,100000.0,951.787298,...,262.590621,2000000.0,80,2.451599e+05,0.040000,0.875000,0.607518,0.604999,54.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18195,99.0,12.965957,0.038599,293.156396,0.000678,0.375708,0.001146,0.000200,100000.0,1394.531458,...,290.889510,15000000.0,100,1.633284e+06,0.047029,0.987254,0.898726,0.325955,45.0,45.0
18196,99.0,12.981516,0.038991,293.156396,0.000686,0.384297,0.001128,0.000200,100000.0,2154.226023,...,286.025116,15000000.0,100,1.633284e+06,0.047029,0.987254,0.898726,0.325955,45.0,45.0
18197,99.0,12.983409,0.020000,293.305418,0.000616,0.101148,0.001126,0.000200,100000.0,298.759861,...,297.013400,15000000.0,100,1.633284e+06,0.047029,0.987254,0.895262,0.328554,45.0,45.0
18198,99.0,12.972423,0.027499,293.177894,0.000582,0.190891,0.001127,0.000200,100000.0,398.306920,...,296.446248,15000000.0,100,1.633284e+06,0.047029,0.987254,0.898319,0.326261,45.0,45.0


In [17]:
combined_bias=np.abs(np.sum(y,axis=1))   # getting combined bias over all cases for all model parameter samples using just this folds training data
best_idx=np.argmin(combined_bias)

y_fold = y[X['sample'] == best_idx] # reducing target to values for this specific sample
X_fold = X[X['sample'] == best_idx].copy() # reducing features to values for this specific sample
# parameters.append({
#     'ss_alpha': X_fold['ss_alpha'].values[0],
#     'k_b': X_fold['k_b'].values[0]
# })
# X_fold = X_fold.drop(columns=['sample', 'ss_alpha', 'k_b'], axis=1)

In [31]:
sample_loc= (X['sample'] == 0)
y_default=y[sample_loc]

In [32]:
y_default

,model_bias_cap
0,0.046460
1,0.055711
2,0.154376
3,0.221871
4,-0.102937
...,...
177,0.318433
178,0.236120
179,0.431803
180,0.381430


In [ ]:
df = xr_data.to_dataframe().reset_index()
        # df['log_z0'] = np.log(df['z0'].values)

X = df.drop(['sample', 'case_index', 'wind_farm', 'flow_case','model_bias_cap',
                'wind_direction','ss_alpha','k_b', 'pw_power_cap', 'ref_power_cap'], axis=1)#,'nt','turb_rated_power'
y = df[['model_bias_cap']]
validation_data=df[['turb_rated_power', 'pw_power_cap', 'ref_power_cap']]
groups=df['wind_farm']

In [ ]:
logo = LeaveOneGroupOut()
for train_idx, test_idx in logo.split(X, y, groups):
    print("TEST FARMS:", groups.values[test_idx], "TRAIN:", groups.values[train_idx])


TEST: ['HR1' 'HR1' 'HR1' 'HR1' 'HR1' 'HR1' 'HR1'] TRAIN: ['HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2'
 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR2' 'HR3' 'HR3' 'HR3'
 'HR3' 'HR3' 'HR3' 'HR3' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1'
 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1'
 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1' 'NYSTED1'
 'NYSTED1' 'NYSTED1' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2'
 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2'
 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2' 'NYSTED2'
 'NYSTED2' 'NYSTED2' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10'
 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA10' 'VirtWF_ABL_IEA15_ali_DX5_DY5'
 'Virt